In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import h5pyd
import dateutil
from pyproj import Proj

In [4]:
from mmctools.helper_functions import theta

# WIND Toolkit analysis
written by Eliot Quon (eliot.quon@nrel.gov)

In [5]:
t0 = pd.to_datetime('2007-01-01')

In [6]:
# lat/lon from Google maps
ref_coords = (42.921494, -105.785106)

In [7]:
compressed_output = 'data'

if not os.path.isdir(compressed_output):
    os.makedirs(compressed_output)

## 1. load data
output properly formatted dataframe

In [8]:
# Open the wind data "file"
# server endpoint, username, password is found via a config file
f = h5pyd.File("/nrel/wtk-us.h5", 'r')  

In [9]:
list(f.attrs)

['history']

In [10]:
f.attrs['history']

'Produced by 3TIER, Inc. under NREL subcontract AGV-2-22460-01'

In [11]:
fields = list(f)
fields

['inversemoninobukhovlength_2m',
 'status',
 'windspeed_10m',
 'temperature_80m',
 'temperature_160m',
 'temperature_200m',
 'pressure_200m',
 'DIF',
 'temperature_10m',
 'winddirection_120m',
 'windspeed_120m',
 'windspeed_140m',
 'temperature_60m',
 'relativehumidity_2m',
 'windspeed_200m',
 'temperature_140m',
 'precipitationrate_0m',
 'winddirection_160m',
 'pressure_0m',
 'GHI',
 'windspeed_80m',
 'winddirection_100m',
 'temperature_2m',
 'temperature_40m',
 'coordinates',
 'winddirection_60m',
 'windspeed_160m',
 'winddirection_40m',
 'winddirection_10m',
 'DNI',
 'winddirection_200m',
 'windspeed_60m',
 'datetime',
 'pressure_100m',
 'windspeed_40m',
 'temperature_120m',
 'windspeed_100m',
 'winddirection_140m',
 'temperature_100m',
 'winddirection_80m']

### - parse datetime    

In [12]:
%%time
dt = pd.DataFrame({'datetime': f['datetime'][:]})
dt['datetime'] = dt['datetime'].apply(dateutil.parser.parse)
dt.head()

CPU times: user 1.99 s, sys: 13 ms, total: 2 s
Wall time: 3.13 s


In [13]:
# not sure what this status is reporting...
status = f['status'][:]
assert np.all(status == 0)

### - get coordinates nearest to reference

In [14]:
# from hsds-examples/notebooks/01_WTK_introduction.ipynb
def indicesForCoord(f, lat_index, lon_index):
    dset_coords = f['coordinates']
    projstring = """+proj=lcc +lat_1=30 +lat_2=60 
                    +lat_0=38.47240422490422 +lon_0=-96.0 
                    +x_0=0 +y_0=0 +ellps=sphere 
                    +units=m +no_defs """
    projectLcc = Proj(projstring)
    origin_ll = reversed(dset_coords[0][0])  # Grab origin directly from database
    origin = projectLcc(*origin_ll)
    
    coords = (lon_index,lat_index)
    coords = projectLcc(*coords)
    delta = np.subtract(coords, origin)
    ij = [int(round(x/2000)) for x in delta]
    return tuple(reversed(ij))

In [15]:
idx = indicesForCoord(f, *ref_coords)

In [16]:
f['coordinates']

<HDF5 dataset "coordinates": shape (1602, 2976), type "|V16">

In [17]:
actual_coords = f['coordinates'][idx[0],idx[1]]
print('Reference location:',ref_coords)
print('Actual grid location:',actual_coords)

Reference location: (42.921494, -105.785106)
Actual grid location: (42.927273, -105.770126)


### - extract time-height data

In [18]:
timeheight_fields = [
    'windspeed',      # [m/s]
    'winddirection',  # [deg]
    'pressure',       # [Pa]
    'temperature',    # [K]
]

In [19]:
# first check data and initialized dataframe
heights = []
for name in timeheight_fields:
    fullnames = [field for field in fields if field.startswith(name+'_')]
    print(name,fullnames)
    assert all([fullname.endswith('m') for fullname in fullnames])
    heights += [float(fullname.split('_')[-1][:-1]) for fullname in fullnames]
heights = np.unique(heights)

timeindex = pd.Index(dt['datetime'])
multicolumns = pd.MultiIndex.from_product((timeheight_fields, heights), names=(None,'height'))
df = pd.DataFrame(index=timeindex, columns=multicolumns, dtype=float)

windspeed ['windspeed_10m', 'windspeed_120m', 'windspeed_140m', 'windspeed_200m', 'windspeed_80m', 'windspeed_160m', 'windspeed_60m', 'windspeed_40m', 'windspeed_100m']
winddirection ['winddirection_120m', 'winddirection_160m', 'winddirection_100m', 'winddirection_60m', 'winddirection_40m', 'winddirection_10m', 'winddirection_200m', 'winddirection_140m', 'winddirection_80m']
pressure ['pressure_200m', 'pressure_0m', 'pressure_100m']
temperature ['temperature_80m', 'temperature_160m', 'temperature_200m', 'temperature_10m', 'temperature_60m', 'temperature_140m', 'temperature_2m', 'temperature_40m', 'temperature_120m', 'temperature_100m']


In [20]:
%%time
# now fill in dataframe
for name in timeheight_fields:
    fullnames = [field for field in fields if field.startswith(name+'_')]
    heights = [float(fullname.split('_')[-1][:-1]) for fullname in fullnames]
    for fullname,height in zip(fullnames,heights):
        print('Reading',name,height)
        df.loc[:,(name,height)] = f[fullname][:,idx[0],idx[1]]
df = df.stack(dropna=False)

# CPU times: user 955 ms, sys: 267 ms, total: 1.22 s
# Wall time: 5min 15s

Reading windspeed 10.0
Reading windspeed 120.0
Reading windspeed 140.0
Reading windspeed 200.0
Reading windspeed 80.0
Reading windspeed 160.0
Reading windspeed 60.0
Reading windspeed 40.0
Reading windspeed 100.0
Reading winddirection 120.0
Reading winddirection 160.0
Reading winddirection 100.0
Reading winddirection 60.0
Reading winddirection 40.0
Reading winddirection 10.0
Reading winddirection 200.0
Reading winddirection 140.0
Reading winddirection 80.0
Reading pressure 200.0
Reading pressure 0.0
Reading pressure 100.0
Reading temperature 80.0
Reading temperature 160.0
Reading temperature 200.0
Reading temperature 10.0
Reading temperature 60.0
Reading temperature 140.0
Reading temperature 2.0
Reading temperature 40.0
Reading temperature 120.0
Reading temperature 100.0
CPU times: user 783 ms, sys: 304 ms, total: 1.09 s
Wall time: 4min 53s


In [21]:
df

pressure  temperature  winddirection  \
datetime            height                                             
2007-01-01 00:00:00 0.0     83614.609375          NaN            NaN   
                    2.0              NaN   270.053711            NaN   
                    10.0             NaN   270.596954     252.978912   
                    40.0             NaN   270.929596     249.067657   
                    60.0             NaN   270.844147     239.586166   
                    80.0             NaN   270.679352     226.962494   
                    100.0   82555.312500   270.468750     219.118011   
                    120.0            NaN   270.279541     212.394180   
                    140.0            NaN   270.090332     206.483353   
                    160.0            NaN   269.937744     201.907410   
                    200.0   81527.148438   269.724121     196.842560   
2007-01-01 01:00:00 0.0     83606.375000          NaN            NaN   
                    2.0              NaN   269.681396            NaN   
                    10.0             NaN   269.922485     113.755913   
                    40.0             NaN   270.166626     122.023376   
                    60.0             NaN   270.102539     126.533401   
                    80.0             NaN   269.962158     129.203156   
                    100.0   82544.328125   270.044556     137.838684   
                    120.0            NaN   270.133057     144.732819   
                    140.0            NaN   270.239868     150.006409   
                    160.0            NaN   270.337524     154.027527   
                    200.0   81516.156250   270.548126     163.470566   
2007-01-01 02:00:00 0.0     83563.343750          NaN            NaN   
                    2.0              NaN   269.763794            NaN   
                    10.0             NaN   269.803467      94.721519   
                    40.0             NaN   269.998779     109.377731   
                    60.0             NaN   269.998779     117.343063   
                    80.0             NaN   269.943848     122.644119   
                    100.0   82501.296875   270.004883     128.912018   
                    120.0            NaN   270.133057     135.992920   
...                                  ...          ...            ...   
2013-12-31 21:00:00 40.0             NaN   275.775940     271.694702   
                    60.0             NaN   275.650818     271.590332   
                    80.0             NaN   275.528748     271.562866   
                    100.0   82098.453125   275.391388     271.639771   
                    120.0            NaN   275.269318     271.722168   
                    140.0            NaN   275.150299     271.821045   
                    160.0            NaN   275.040436     271.875977   
                    200.0   81094.085938   274.848175     271.804565   
2013-12-31 22:00:00 0.0     83117.468750          NaN            NaN   
                    2.0              NaN   275.507385            NaN   
                    10.0             NaN   275.647766     272.112183   
                    40.0             NaN   275.623352     271.672729   
                    60.0             NaN   275.547058     271.282684   
                    80.0             NaN   275.458527     270.865204   
                    100.0   82081.968750   275.348663     270.304871   
                    120.0            NaN   275.247955     269.750061   
                    140.0            NaN   275.138092     269.112823   
                    160.0            NaN   275.025177     268.437134   
                    200.0   81078.523438   274.796295     267.074799   
2013-12-31 23:00:00 0.0     83076.265625          NaN            NaN   
                    2.0              NaN   274.814606            NaN   
                    10.0             NaN   275.070953     268.629425   
                    40.0             NaN   275.214386     267.970215   


In [22]:
%time df.to_csv(os.path.join(compressed_output,'WTK.raw.csv.gz'), compression='gzip')

# CPU times: user 8.51 s, sys: 117 ms, total: 8.63 s
# Wall time: 8.72 s

CPU times: user 8.91 s, sys: 142 ms, total: 9.06 s
Wall time: 9.09 s


### - extract column data

In [23]:
%time z_L = 2.0 * f['inversemoninobukhovlength_2m'][:,idx[0],idx[1]]

CPU times: user 16.8 ms, sys: 4.18 ms, total: 21 ms
Wall time: 9.15 s


In [24]:
# global horizontal irradiance
%time GHI = f['GHI'][:,idx[0],idx[1]]  # [W/m^2]

CPU times: user 17.7 ms, sys: 4.73 ms, total: 22.4 ms
Wall time: 9.13 s


In [25]:
# if we try to calculate virtual temperatures, then we'll have to assume that this is approximately constant with height
%time RH = f['relativehumidity_2m'][:,idx[0],idx[1]]  # [%]?

CPU times: user 17.4 ms, sys: 4.65 ms, total: 22 ms
Wall time: 9.59 s


In [26]:
%time precip = f['precipitationrate_0m'][:,idx[0],idx[1]]  # [ ]?

CPU times: user 17.3 ms, sys: 4.14 ms, total: 21.5 ms
Wall time: 9.37 s


In [27]:
surf = pd.DataFrame({'z/L': z_L,
                     'RH': RH,
                     'precipitationrate': precip,
                     'GHI': GHI,
                     'pressure': df['pressure'].xs(0,level='height'),
                    },
                    index=timeindex)

In [28]:
surf.tail()

,z/L,RH,precipitationrate,GHI,pressure
datetime,,,,,
2013-12-31 19:00:00,0.000000,62.541580,0.0,416.234314,83182.468750
2013-12-31 20:00:00,0.000610,62.247841,0.0,397.488312,83128.453125
2013-12-31 21:00:00,0.001831,65.359818,0.0,306.459229,83132.117188
2013-12-31 22:00:00,0.006714,66.294746,0.0,136.051514,83117.468750
2013-12-31 23:00:00,0.012207,68.063904,0.0,18.150879,83076.265625


In [29]:
%time surf.to_csv(os.path.join(compressed_output,'WTK_surf.raw.csv.gz'), compression='gzip')

CPU times: user 684 ms, sys: 10.5 ms, total: 694 ms
Wall time: 694 ms


## 2. calculate/interpolate additional quantities

### - fill in pressure levels
Note: spot checking pressure profiles up to 200-m a.g.l. shows that the pressure profile is indeed linear

In [30]:
p = df['pressure'].unstack(level=0)

In [31]:
%time p = p.interpolate(method='index', limit=None, limit_area='inside')

# CPU times: user 16.7 s, sys: 80.4 ms, total: 16.8 s
# Wall time: 16.7 s

CPU times: user 17.4 s, sys: 66.3 ms, total: 17.5 s
Wall time: 17.4 s


In [32]:
df['pressure'] = p.stack().reorder_levels(['datetime','height']).sort_index()

### - calculate potential temperature

In [33]:
df['theta'] = theta(df['temperature'], df['pressure'], p0=df['pressure'].xs(0,level='height'))

In [34]:
df[['pressure','temperature','theta']]

pressure  temperature       theta
datetime            height                                       
2007-01-01 00:00:00 0.0     83614.609375          NaN         NaN
                    2.0     83593.423437   270.053711  270.073284
                    10.0    83508.679688   270.596954  270.695079
                    40.0    83190.890625   270.929596  271.323542
                    60.0    82979.031250   270.844147  271.435847
                    80.0    82767.171875   270.679352  271.469102
                    100.0   82555.312500   270.468750  271.456794
                    120.0   82349.679688   270.279541  271.460450
                    140.0   82144.046875   270.090332  271.464457
                    160.0   81938.414062   269.937744  271.505651
                    200.0   81527.148438   269.724121  271.681485
2007-01-01 01:00:00 0.0     83606.375000          NaN         NaN
                    2.0     83585.134063   269.681396  269.700995
                    10.0    83500.170313   269.922485  270.020629
                    40.0    83181.556250   270.166626  270.560524
                    60.0    82969.146875   270.102539  270.694218
                    80.0    82756.737500   269.962158  270.751952
                    100.0   82544.328125   270.044556  271.033731
                    120.0   82338.693750   270.133057  271.316036
                    140.0   82133.059375   270.239868  271.617495
                    160.0   81927.425000   270.337524  271.910525
                    200.0   81516.156250   270.548126  272.514303
2007-01-01 02:00:00 0.0     83563.343750          NaN         NaN
                    2.0     83542.102813   269.763794  269.783409
                    10.0    83457.139063   269.803467  269.901618
                    40.0    83138.525000   269.998779  270.392636
                    60.0    82926.115625   269.998779  270.590537
                    80.0    82713.706250   269.943848  270.733997
                    100.0   82501.296875   270.004883  270.994426
                    120.0   82296.029688   270.133057  271.316305
...                                  ...          ...         ...
2013-12-31 21:00:00 40.0    82718.651563   275.775940  276.169476
                    60.0    82511.918750   275.650818  276.241804
                    80.0    82305.185938   275.528748  276.317650
                    100.0   82098.453125   275.391388  276.378618
                    120.0   81897.579687   275.269318  276.449730
                    140.0   81696.706250   275.150299  276.524348
                    160.0   81495.832812   275.040436  276.608621
                    200.0   81094.085938   274.848175  276.806217
2013-12-31 22:00:00 0.0     83117.468750          NaN         NaN
                    2.0     83096.758750   275.507385  275.527021
                    10.0    83013.918750   275.647766  275.746060
                    40.0    82703.268750   275.623352  276.017441
                    60.0    82496.168750   275.547058  276.138981
                    80.0    82289.068750   275.458527  276.248778
                    100.0   82081.968750   275.348663  276.337683
                    120.0   81881.279688   275.247955  276.430080
                    140.0   81680.590625   275.138092  276.513746
                    160.0   81479.901563   275.025177  276.594800
                    200.0   81078.523438   274.796295  276.755210
2013-12-31 23:00:00 0.0     83076.265625          NaN         NaN
                    2.0     83055.537500   274.814606  274.834219
                    10.0    82972.625000   275.070953  275.169176
                    40.0    82661.703125   275.214386  275.608432
                    60.0    82454.421875   275.214386  275.806409
                    80.0    82247.140625   275.186920  275.977483
                    100.0   82039.859375   275.119781  276.109346
                    120.0   81839.168750   275.046539  276.229267
                    140.0   81638.478125   274.948883  276.325160
      

In [35]:
%time df.drop(index=0,level='height').to_csv(os.path.join(compressed_output,'WTK.interp.csv.gz'), compression='gzip')

# CPU times: user 13.8 s, sys: 177 ms, total: 14 s
# Wall time: 14 s

CPU times: user 13.8 s, sys: 291 ms, total: 14.1 s
Wall time: 14.1 s
